# First Order Motion Model
![Sample of image animation by first order motion model](https://raw.githubusercontent.com/sony/nnabla-examples/master/GANs/first-order-model/imgs/sample_fake.gif)

This example interactively demonstrates [First Order Motion Model](https://papers.nips.cc/paper/2019/file/31c0b36aef265d9221af80872ceb62f9-Paper.pdf), a model for motion transfer which can be used to generate an *image animation*.

**This notebook contains a script (record_video) which is distributed under CC BY-SA 4.0 license by [Emily Xie](https://stackoverflow.com/users/2738225/emily-xie) at https://stackoverflow.com/a/62804023, therefore this notebook is also distributed under the same license.**


# Preparation
Let's start by installing nnabla and accessing [nnabla-examples repository](https://github.com/sony/nnabla-examples). If you're running on Colab, make sure that your Runtime setting is set as GPU, which can be set up from the top menu (Runtime → change runtime type), and make sure to click **Connect** on the top right-hand side of the screen before you start.

In [ ]:
!pip install nnabla-ext-cuda100
!git clone https://github.com/sony/nnabla-examples.git
%cd nnabla-examples/GANs/first-order-model

We need the latest PyYAML package to run this example, so we will upgrade that.

In [ ]:
!pip install --upgrade PyYAML

Now we define a function which plays videos in Colab. Simply run the following cell.

In [ ]:
from IPython.display import HTML
from base64 import b64encode


def play_video(filename, height=512, width=512):
    mp4 = open(filename, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width={width} height={height} controls>
          <source src={data_url} type="video/mp4">
    </video>""")

# Quickstart
Now take a look what this model can do! Simply run the following cells.


In [ ]:
!python animate.py --source imgs/sample_src.png \
                  --driving imgs/sample_drv.mp4 \
                  --adapt-movement-scale --fps 24

In [ ]:
play_video('result/arbitrary/sample_src.png_by_sample_drv.mp4')

As you can see, the model generates an *animation* of source image using the reference motion of driving video.

Next, let's try with a different image. Please upload some face image you have. Note that the face **must** have the same pose as the first frame of the driving video, in short, the image must contain frontal face.

# Upload Image
Run the following cell to upload your own image. Note that too small images might cause a poor result.


In [13]:
from google.colab import files

img = files.upload()

Saving 00_eyes_open.jpeg to 00_eyes_open.jpeg


Let's rename the image for convenience.



In [14]:
import os
ext = os.path.splitext(list(img.keys())[-1])[-1]
os.rename(list(img.keys())[-1], "input_image{}".format(ext)) 
input_img = "input_image" + ext

# Image Animation using arbitrary source images

Now that you uploaded the image, let's use that for animation. The source image comes to life!

In [17]:
!cat animate.py

# Copyright (c) 2020 Sony Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import logging
import argparse
import numpy as np

import nnabla as nn
import nnabla.monitor as nm
import nnabla.functions as F
import nnabla.logger as logger

from nnabla.utils.image_utils import imread
from nnabla.ext_utils import get_extension_context
from nnabla.utils.data_source_loader import download

from utils import read_yaml
from frames_dataset import read_video
from mod

In [18]:
!python animate.py --source $input_img \
                  --driving imgs/sample_drv.mp4 \
                  --adapt-movement-scale --fps 24 \
                  --only-generated

2021-07-13 00:14:33,195 [nnabla][INFO]: Initializing CPU extension...
2021-07-13 00:14:34,088 [nnabla][INFO]: Initializing CUDA extension...
2021-07-13 00:14:34,117 [nnabla][INFO]: Initializing cuDNN extension...
Loading pretrained_fomm_params.h5 for image animation...
100% 125/125 [00:04<00:00, 28.38it/s]


In [19]:
play_video(f'result/arbitrary/{input_img}_by_sample_drv.mp4')

# Record Motion Reference Video
Now is the time to use your motion! Here you can record a video as motion reference and the model will *animate* the source image just like you do! First of all, define the function which allows Colab to access a camera and record the video with it.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode


def record_video(filename='video.mp4'):

    # code from: https://stackoverflow.com/questions/62529304/is-there-any-way-to-capture-live-video-using-webcam-in-google-colab

    js = Javascript("""
        async function recordVideo() {
            const options = { mimeType: "video/webm; codecs=vp9" };
            const div = document.createElement('div');
            const capture = document.createElement('button');
            const stopCapture = document.createElement("button");
            capture.textContent = "Start Recording";
            capture.style.background = "green";
            capture.style.color = "white";

            stopCapture.textContent = "Stop Recording";
            stopCapture.style.background = "red";
            stopCapture.style.color = "white";
            div.appendChild(capture);

            const video = document.createElement('video');
            const recordingVid = document.createElement("video");
            video.style.display = 'block';

            const stream = await navigator.mediaDevices.getUserMedia({video: { width: {ideal: 256}, height: {ideal: 256}, facingMode: 'user'}});
            let recorder = new MediaRecorder(stream, options);
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => {
                capture.onclick = resolve;
            });
            recorder.start();
            capture.replaceWith(stopCapture);
            await new Promise((resolve) => stopCapture.onclick = resolve);
            recorder.stop();

            let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
            let arrBuff = await recData.data.arrayBuffer();
            stream.getVideoTracks()[0].stop();
            div.remove();

            let binaryString = "";
            let bytes = new Uint8Array(arrBuff);
            bytes.forEach((byte) => {
                binaryString += String.fromCharCode(byte);
            })
            return btoa(binaryString);
        }
    """)
    try:
        display(js)
        data = eval_js('recordVideo({})')
        binary = b64decode(data)
        with open(filename, "wb") as video_file:
            video_file.write(binary)
        print("Finished recording video.")
    except Exception as err:
        # In case any exceptions arise
        print(str(err))
    return filename

The following cell tries to use a camera on your device. Press "Start Recording" and it will start recording! When you press "Stop Recording", the recording will stop and the video file will be saved.

In [ ]:
video_width = 256

video_path = record_video()
play_video(video_path, height=256, width=256)

Then let's make an animation using your video!

In [ ]:
!python animate.py --source $input_img \
                  --driving video.mp4 \
                  --adapt-movement-scale --fps 24

Executing the following cell will show you the result.

Note that the animation result strongly depends on the initial pose of source image and driving video, so if the result looks bad, just try with the same pose as the source image. 

In [ ]:
play_video(f'result/arbitrary/{input_img}_by_video.mp4')